In [1]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/coref-model-2018.02.05.tar.gz")

/Users/frederikwarburg/anaconda3/envs/neuralcoref/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [60]:
import pandas as pd
import numpy as np
import spacy
from calculatescore import calculate_score, eval_gender, evaluate

In [3]:
data = pd.read_csv('data/gap/gap-test.tsv', sep = '\t')
data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers


In [23]:
#text = 'This is the first sentence. This is the second sentence.'
prediction = predictor.predict(
  document=text
)
prediction['clusters']

Encountered the antecedent_indices key in the model's return dictionary which couldn't be split by the batch size. Key will be ignored.


[[[1, 1], [8, 8], [12, 12]],
 [[19, 20], [39, 40]],
 [[26, 28], [57, 59]],
 [[57, 58], [61, 61], [65, 65], [69, 69]]]

In [56]:
# NOTE THAT THIS TAKES A LONG TIME > 30 min

results = []
for index in range(len(data['Text'])):
    
    text = data['Text'][index]
    prediction = predictor.predict(
      document=text
    )

    pronoun_span = get_span_from_offset(text, data['Pronoun-offset'][index])
    A_span = get_span_from_offset(text, data['A-offset'][index])
    B_span = get_span_from_offset(text, data['B-offset'][index])

    pronoun_cluster_num = get_cluster_num(pronoun_span, prediction)
    A_cluster_num = get_cluster_num(A_span, prediction)
    B_cluster_num = get_cluster_num(B_span, prediction)

    A, B, N = 0, 0, 0
    for pronoun_pred in pronoun_cluster_num:
        if pronoun_pred in A_cluster_num: A = 1 
        if pronoun_pred in B_cluster_num: B = 1 

    if A == 0 and B == 0: N = 1
    
    results.append([A, B, N])
    
    if index % 10 == 0:
        print(index, " / ", len(data['Text']))


0  /  2000
10  /  2000
20  /  2000
30  /  2000
40  /  2000
50  /  2000
60  /  2000
70  /  2000
80  /  2000
90  /  2000
100  /  2000
110  /  2000
120  /  2000
130  /  2000
140  /  2000
150  /  2000
160  /  2000
170  /  2000
180  /  2000
190  /  2000
200  /  2000
210  /  2000
220  /  2000
230  /  2000
240  /  2000
250  /  2000
260  /  2000
270  /  2000
280  /  2000
290  /  2000
300  /  2000
310  /  2000
320  /  2000
330  /  2000
340  /  2000
350  /  2000
360  /  2000
370  /  2000
380  /  2000
390  /  2000
400  /  2000
410  /  2000
420  /  2000
430  /  2000
440  /  2000
450  /  2000
460  /  2000
470  /  2000
480  /  2000
490  /  2000
500  /  2000
510  /  2000
520  /  2000
530  /  2000
540  /  2000
550  /  2000
560  /  2000
570  /  2000
580  /  2000
590  /  2000
600  /  2000
610  /  2000
620  /  2000
630  /  2000
640  /  2000
650  /  2000
660  /  2000
670  /  2000
680  /  2000
690  /  2000
700  /  2000
710  /  2000
720  /  2000
730  /  2000
740  /  2000
750  /  2000
760  /  2000
770  /  20

In [57]:
# Random guess
predictions = pd.DataFrame(results, columns=['A','B','NEITHER'])
predictions.to_csv('data/predictions_allen_default1.csv')
predictions.head()

,A,B,NEITHER
0,0,0,1
1,1,0,0
2,0,1,0
3,0,0,1
4,1,0,0


In [65]:
calculate_score('data/right_answers.csv','data/predictions_allen_default1.csv')

21.476225184070334

In [66]:
precision, recall, f1 = evaluate('data/right_answers.csv', 'data/predictions_allen_default1.csv')
print(precision, recall, f1)

[0.43140244 0.47359736 0.10805085] [0.32379863 0.31027027 0.50746269] [0.36993464 0.37491835 0.17816594]


In [67]:
# Gold path
path_gold = 'data/right_answers.csv'

# Path full data #
path_full = 'data/gendered-pronoun-resolution/test_stage_1.tsv'
df = pd.read_csv(path_full, sep = '\t', index_col = 0) 

# Create predictions #
predictions = pd.DataFrame(results, columns=['A','B','NEITHER'], index = list(df.index))
predictions.to_csv('data/predictions_allen_default2.csv')
path_pred = 'data/predictions_allen_default2.csv'

male_sc, female_sc = eval_gender(path_gold, path_pred, path_full)

Male precision: A = 0.4281, B = 0.4573, Neither = 0.1032
Male recall: A = 0.2990867579908676, B = 0.32679738562091504, Neither = 0.46601941747572817
Male F1: A = 0.3522, B = 0.3812, Neither = 0.169 

Female precision: A = 0.4342857142857143, B = 0.49280575539568344, Neither = 0.1127348643006263
Female recall: A = 0.3486238532110092, B = 0.2939914163090129, Neither = 0.5510204081632653
Female F1: A = 0.3868, B = 0.3683, Neither = 0.1872
